In [3]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score

In [4]:
df_rating=pd.read_csv("BX-Book-Ratings.csv", encoding="latin1")
df_rating.head()

,user_id,isbn,rating
0,276725,034545104X,0
1,276726,155061224,5
2,276727,446520802,0
3,276729,052165615X,3
4,276729,521795028,6


In [5]:
df_rating.shape

(1048575, 3)

In [6]:
df_rating.sort_values(['user_id','isbn'],inplace=True)
df_rating=df_rating.head(10000)
df_rating.reset_index()
df_rating.head()

,user_id,isbn,rating
9561,2,195153448,0
9562,7,34542252,0
9571,8,074322678X,5
9574,8,080652121X,0
9576,8,1552041778,5


In [7]:
from sklearn.preprocessing import LabelEncoder
ll=LabelEncoder()

In [8]:
df_rating['isbn']=ll.fit_transform(df_rating['isbn'])
df_rating.head()

,user_id,isbn,rating
9561,2,1467,0
9562,7,2490,0
9571,8,443,5
9574,8,475,0
9576,8,877,5


In [9]:
df_books=pd.read_csv("BX-Books.csv", encoding="latin1")
df_users = pd.read_csv("BX-Users.csv", encoding="latin1")
df_books.head()
df_users.head()

C:\Users\HP\Downloads\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\HP\Downloads\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,user_id,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [10]:
n_users = df_rating["user_id"].unique().shape[0]
n_books = df_rating["isbn"].unique().shape[0]
print(n_users,n_books)

1323 8742


In [11]:
data_matrix = np.zeros((n_users, n_books))
for line in df_rating.head().itertuples():
    #print(line)
    data_matrix[line[1]-1, line[2]-1] = line[3]
data_matrix.shape

(1323, 8742)

In [12]:
from sklearn.metrics.pairwise import pairwise_distances
user_sim=pairwise_distances(data_matrix,metric='cosine')
item_sim=pairwise_distances(data_matrix.T,metric='cosine')

In [13]:
def predict(ratings, similarity, type='user'):
    pred=0
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #We use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [14]:
user_prediction = predict(data_matrix, user_sim, type='user')
item_prediction = predict(data_matrix, item_sim, type='item')

In [15]:
from sklearn.metrics import mean_squared_error
mse=mean_squared_error

In [16]:
def rmse_new(y_trues,y_preds):
    y_preds=y_preds[y_trues.nonzero()].flatten()
    y_trues=y_trues[y_trues.nonzero()].flatten()
    return mean_squared_error(y_trues,y_preds,squared=False)

UBCF

In [17]:
rmse_new(data_matrix,user_prediction)

4.9988560970029745

IBCF

In [18]:
rmse_new(data_matrix,item_prediction)

5.0